<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/English/tpu_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# a = []
# while(1):
#     a.append('1111111111111111111111111111111111111111111111')

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [1]:
!pip install tensorflow==2.1.0
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()

In [0]:
import pandas as pd
import os
import numpy as np
#import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Flatten, Input
from tensorflow.keras.optimizers import RMSprop,Adam,Adadelta

# Load data

In [4]:
train=pd.read_csv( '/content/drive/My Drive/dataset/OffenseEval2020/data/English/task_a_distant.tsv',sep="\t")
train['average']=train['average'].values
train['average']=train['average'].round().astype('int')
train.head()

,id,text,average,std
0,1159533701283352576,"First time I heard his name in camp, he seems ...",0,0.187379
1,1159533703522992128,When I go to drink with Tsubaki he would alway...,0,0.145998
2,1159533703758061570,@USER His ass need to stay up 😂😂,1,0.140628
3,1159533703904800769,most important tweet of the day : Fuck Donald ...,1,0.187498
4,1159533705125343232,You wanna leave? then feel free cus I promise ...,1,0.097098


In [5]:
len(train)

9075418

In [6]:
train['average'].value_counts()

0    7628650
1    1446768
Name: average, dtype: int64

In [7]:
train=train.groupby('average',as_index=False).apply(lambda x: x.sample(1446580))
train['average'].value_counts()

1    1446580
0    1446580
Name: average, dtype: int64

In [8]:
val=pd.read_csv( '/content/drive/My Drive/dataset/OLID/data/olid-training-v1.0.tsv',sep="\t")

val.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [9]:
test=pd.read_csv( '/content/drive/My Drive/dataset/OffenseEval2020/data/English/test_a_tweets.tsv',sep="\t")
test.head()

,id,tweet
0,A0,¿Who the fuck is Yoru?
1,A1,@USER @USER He's an evil law breaker that shou...
2,A2,Now hiring for 49 #Labor job opportunities in ...
3,A3,"#NerkondaPaarvai' - As Of Now, There Is NO Ear..."
4,A4,"@USER Hahahaha I wish...but a week is good, I’..."


In [10]:
y_train=train['average']
train=train['text']
train.head()

0  4445184                             @USER So sad and scary 😖
   6830427    I hope they had a refundable deposit on the st...
   758808     @USER I know! I have 3 weeks off before my new...
   274981     @USER Leaping lordy, Devin, why do tweet? You ...
   7598283    FatKidDeals will post a deal for 650 Twizzlers...
Name: text, dtype: object

In [11]:
y_val=val['subtask_a']
val=val['tweet']
val.head()

0    @USER She should ask a few native Americans wh...
1    @USER @USER Go home you’re drunk!!! @USER #MAG...
2    Amazon is investigating Chinese employees who ...
3    @USER Someone should'veTaken" this piece of sh...
4    @USER @USER Obama wanted liberals &amp; illega...
Name: tweet, dtype: object

In [12]:
ids=test['id']
test=test['tweet']
test.head()

0                               ¿Who the fuck is Yoru?
1    @USER @USER He's an evil law breaker that shou...
2    Now hiring for 49 #Labor job opportunities in ...
3    #NerkondaPaarvai' - As Of Now, There Is NO Ear...
4    @USER Hahahaha I wish...but a week is good, I’...
Name: tweet, dtype: object

In [13]:

train=train.str.replace('\d+', '')
train=train.str.replace('@USER', "") 
train=train.str.replace('URL', '')
train= train.str.lower()

train.head()

0  4445184                                   so sad and scary 😖
   6830427    i hope they had a refundable deposit on the st...
   758808      i know! i have  weeks off before my new job.....
   274981      leaping lordy, devin, why do tweet? you must ...
   7598283    fatkiddeals will post a deal for  twizzlers fo...
Name: text, dtype: object

In [14]:

val=val.str.replace('\d+', '')
val=val.str.replace('@USER', "") 
val=val.str.replace('URL', '')
val= val.str.lower()
val.head()

0     she should ask a few native americans what th...
1          go home you’re drunk!!!  #maga #trump 👊🇺🇸👊 
2    amazon is investigating chinese employees who ...
3     someone should'vetaken" this piece of shit to...
4      obama wanted liberals &amp; illegals to move...
Name: tweet, dtype: object

In [15]:

test=test.str.replace('\d+', '')
test=test.str.replace('@USER', "") 
test=test.str.replace('URL', '')
test= test.str.lower()
test.head()

0                               ¿who the fuck is yoru?
1      he's an evil law breaker that should be in p...
2    now hiring for  #labor job opportunities in #m...
3    #nerkondapaarvai' - as of now, there is no ear...
4     hahahaha i wish...but a week is good, i’m ext...
Name: tweet, dtype: object

#Label Encoding

In [16]:
len(train),len(val),len(test)

(2893160, 13240, 3887)

In [17]:
le=LabelEncoder()
y_val=le.fit_transform(y_val)
print(len(y_val))

13240


In [18]:
import gc
gc.collect()

63

#BERT

In [3]:
!pip install sentencepiece
!pip install tensorflow-hub

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    
    return model

In [6]:
%%time
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"


CPU times: user 66 ms, sys: 7.95 ms, total: 74 ms
Wall time: 73.5 ms


In [7]:
# TF 2.0
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_host(resolver.master())
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  bert_layer = hub.KerasLayer(module_url, trainable=True)
  model = build_model(bert_layer, max_len=100)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy])


INFO:tensorflow:Initializing the TPU system: 10.63.43.202:8470
INFO:tensorflow:Clearing out eager caches
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorfl

InvalidArgumentError: ignored

In [30]:
!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

AttributeError: ignored

In [0]:
train_input = bert_encode(train.values[0:10000], tokenizer, max_len=100)



In [0]:
val_input = bert_encode(val.values, tokenizer, max_len=100)
test_input = bert_encode(test.values, tokenizer, max_len=100)

In [0]:
len(train_input),len(val_input),len(test_input)

In [0]:
bert_model = build_model(bert_layer, max_len=100)
bert_model.summary()

In [29]:
train_history = bert_model.fit(
    train_input,np.array(y_train),
    validation_data=(val_input,y_val),
    epochs=1,
    batch_size=12)


Train on 2893160 samples, validate on 13240 samples
  54840/2893160 [..............................] - ETA: 94:42:55 - loss: 0.1782 - accuracy: 0.9250

KeyboardInterrupt: ignored

In [0]:
tf.keras.backend.clear_session()